In [ ]:
#This script is for generating XMLs for continuous phylogeography in BEAST with empirical tree sets
#Must provide the paths to your root directory
#As well as a directory of alignment (fastas), latlong tsvs, and empirical trees
#Must also provide a template XML
#fastas, tree files, and latlong files must have matching prefixes to be processed together

In [ ]:
import os
import re
import csv
from Bio import SeqIO
import shutil

In [ ]:
# === Set these paths ===
input_root = "test_looped_xml_gen"
alignment_dir = os.path.join(input_root, "alignment")
latlong_dir = os.path.join(input_root, "latlong_tsv")
tree_dir = os.path.join(input_root, "emptree")
template_file = "test_looped_xml_gen/contphyl_emptre_template.xml"
output_dir = "test_looped_xml_gen/output"


In [6]:
# Collect files grouped by prefix
files_by_prefix = {}


def extract_prefix(filename):
    base = os.path.splitext(filename)[0]
    match = re.match(r"(.*?_\d+)_", base)
    return match.group(1) if match else base

def register_file(path, ext_type):
    prefix = extract_prefix(os.path.basename(path))
    if prefix not in files_by_prefix:
        files_by_prefix[prefix] = {"fasta": None, "tsv": None, "tree": None}
    files_by_prefix[prefix][ext_type] = path

# Look in each specific directory
for fname in os.listdir(alignment_dir):
    if fname.endswith((".fasta", ".fa")):
        register_file(os.path.join(alignment_dir, fname), "fasta")

for fname in os.listdir(latlong_dir):
    if fname.endswith(".tsv"):
        register_file(os.path.join(latlong_dir, fname), "tsv")

for fname in os.listdir(tree_dir):
    if fname.endswith(".trees"):
        register_file(os.path.join(tree_dir, fname), "tree")


# Now process each prefix group
for prefix, paths in files_by_prefix.items():
    fasta_file = paths["fasta"]
    tsv_file = paths["tsv"]
    tree_file = paths["tree"]

    if not fasta_file or not tsv_file or not tree_file:
        print(f"Skipping {prefix}: Missing fasta, tsv, or tree file")
        continue

    print(f"Processing {prefix}")

    # Load FASTA records
    records = list(SeqIO.parse(fasta_file, "fasta"))
    taxa = [r.id for r in records]

    # Load location data from TSV
    location_data = {}
    with open(tsv_file, newline='') as tsvfile:
        reader = csv.DictReader(tsvfile, delimiter='\t', fieldnames=['taxon_id', 'latitude', 'longitude'])
        for row in reader:
            taxon_id = row['taxon_id']
            try:
                decimal_date = taxon_id.split("|")[1]
            except IndexError:
                print(f"Warning: Invalid taxon_id format: {taxon_id}")
                continue
            location_data[taxon_id] = {
                "decimal_date": decimal_date,
                "latitude": row["latitude"],
                "longitude": row["longitude"]
            }

    # Build taxa block
    taxa_block = ['\t<taxa id="taxa">']
    for taxon in taxa:
        if taxon not in location_data:
            print(f"Warning: No location found for {taxon}")
            continue
        data = location_data[taxon]
        taxa_block.append(f'\t\t<taxon id="{taxon}">')
        taxa_block.append(f'\t\t\t<date value="{data["decimal_date"]}" direction="forwards" units="years"/>')
        taxa_block.append(f'\t\t\t<attr name="LATITUDE">\n\t\t\t\t{data["latitude"]}\n\t\t\t</attr>')
        taxa_block.append(f'\t\t\t<attr name="LONGITUDE">\n\t\t\t\t{data["longitude"]}\n\t\t\t</attr>')
        taxa_block.append(f'\t\t\t<!-- START Multivariate diffusion model -->')
        taxa_block.append(f'\t\t\t<attr name="location">\n\t\t\t\t{data["latitude"]} {data["longitude"]}\n\t\t\t</attr>')
        taxa_block.append(f'\t\t\t<!-- END Multivariate diffusion model -->')
        taxa_block.append(f'\t\t</taxon>')
    taxa_block.append('\t</taxa>')
    taxa_block_str = "\n".join(taxa_block)

    # Build alignment block
    alignment_block = ['\t<alignment id="alignment" dataType="nucleotide">']
    for record in records:
        alignment_block.append(f'\t\t<sequence>')
        alignment_block.append(f'\t\t\t<taxon idref="{record.id}"/>')
        alignment_block.append(f'\t\t\t{str(record.seq)}')
        alignment_block.append(f'\t\t</sequence>')
    alignment_block.append('\t</alignment>')
    alignment_block_str = "\n".join(alignment_block)

    # Read template
    with open(template_file, 'r') as f:
        template_xml = f.read()

    # Replace taxa block
    template_xml = re.sub(
        r'<taxa id="taxa">.*?</taxa>',
        taxa_block_str,
        template_xml,
        flags=re.DOTALL
    )

    # Replace empirical tree filename with the correct file name
    tree_filename = os.path.basename(tree_file)
    template_xml = re.sub(
        r'(<empiricalTreeDistributionModel id="treeModel" fileName=")[^"]+(")',
        rf'\1{tree_filename}\2',
        template_xml
    )

    # Replace alignment block
    template_xml = re.sub(
        r'<alignment id="alignment" dataType="nucleotide">.*?</alignment>',
        alignment_block_str,
        template_xml,
        flags=re.DOTALL
    )

    # Replace all ntax=NUM with actual number of sequences
    template_xml = re.sub(r'ntax=\d+', f'ntax={len(records)}', template_xml)

    # Create output directory for this prefix
    prefix_out_dir = os.path.join(output_dir, prefix)
    os.makedirs(prefix_out_dir, exist_ok=True)

    # Write output xml
    output_xml_path = os.path.join(prefix_out_dir, f"{prefix}.xml")
    with open(output_xml_path, 'w') as f:
        f.write(template_xml)

    print(f"Written {output_xml_path}")

    #Copy empirical tree to output folder
    tree_dest = os.path.join(prefix_out_dir, os.path.basename(tree_file))
    shutil.copy(tree_file, tree_dest)
    print(f"Copied empirical tree to {tree_dest}")

print("Batch processing complete!")


Processing Wisconsin_64
Written test_looped_xml_gen/output/Wisconsin_64/Wisconsin_64.xml
Copied empirical tree to test_looped_xml_gen/output/Wisconsin_64/Wisconsin_64_alignment_allWIdt_w_meta.trees
Processing Wisconsin_120
Written test_looped_xml_gen/output/Wisconsin_120/Wisconsin_120.xml
Copied empirical tree to test_looped_xml_gen/output/Wisconsin_120/Wisconsin_120_alignment_allWIdt_w_meta.trees
Batch processing complete!
